# 1. 系统参数

$l = 0.085 m$  
$l_b= 0.075 m$  
$m_b= 0.419 kg$  
$m_w =0.204 kg$   
$I_b= 3.34 × 10^{−3} kg · m^2$   
$I_w= 0.57 × 10^{−3} kg · m^2$   
$C_b= 1.02 × 10^{−3} kg · m^2 · s^{−1}$  
$C_w= 0.05 × 10^{−3} kg · m^2 · s^{−1}$  
$K_m = 25.1 × 10^{−3} Nm · A^{−1}$  
$g = 9.81 m · s^{−2}$

In [26]:
l = 0.085
l_b= 0.075
m_b= 0.419
m_w =0.204 
I_b= 3.34 * 10e-3
I_w= 0.57 * 10e-3
C_b= 1.02 * 10e-3
C_w= 0.05 * 10e-3
K_m = 25.1 * 10e-3
g=9.81

theta_w_d_max = 1000
elastic_coeff = 0.3

dt = 0.01
fps = 1/dt

# 2. 系统动力学方程

$$\ddot{\theta}_b = \frac{(m_b l_b+m_w l)\cdot g\cdot \sin(\theta_b)-T_m-C_w (\dot{\theta}_b-\dot{\theta}_w)}{I_b+m_w l^2}$$
$$\ddot{\theta}_w = \frac{T_m-C_w (\dot{\theta}_w-\dot{\theta}_b)}{I_w}$$
$$T_m=K_m u(t)$$

$$\dot{x}(t)=Ax(t)+Bu(t)$$
$$x(t)=[\theta_b,\dot{\theta}_b,\dot{\theta}_w]^T$$

在$x(t)=[0,0,0]$附近对A线性化：

$$A=\left[\begin{matrix} 
0 & 1 & 0\\
\frac{(m_b l_b+m_w l)\cdot g}{I_b+m_w l^2} & -\frac{C_w}{I_b+m_w l^2} & \frac{C_w}{I_b+m_w l^2}\\
0 & \frac{C_w}{I_w} & -\frac{C_w}{I_w} 
\end{matrix}\right]$$

$$B=\left[\begin{matrix} 
0\\
-K_m\\
K_m\\
\end{matrix}\right]$$

# 3. LQR参数矩阵设定
令:
$$Q=\left[\begin{matrix} 
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{matrix}\right]$$

$$R=\left[\begin{matrix} 
1 
\end{matrix}\right]$$

# 4. LQR控制律求解

In [27]:
import numpy as np
import scipy
A = np.array([[0,1,0],[(m_b*l_b+m_w*l)*g/(I_b+m_w*l**2), -C_w/(I_b+m_w*l**2), C_w/(I_b+m_w*l**2)],[0,C_w/I_w,-C_w/I_w]])
B = np.array([[0],[-K_m],[K_m]])
print(f"A=\n{A}\nB=\n{B}")
Q = np.eye(3)
R = np.array([[1]])
print(f"Q=\n{Q}\nR=\n{R}")

def fun(P):
    return A.T@P + P@A+Q - P@B@np.linalg.inv(R)@B.T@P

P = np.eye(3)
P = scipy.linalg.solve_continuous_are(A, B, Q, R)
print(f"P=\n{P}")

K = np.linalg.inv(R)@B.T@P

print(f"K=\n{K}")

def LQR_controller(x):
    return (-K@x).squeeze()

A=
[[ 0.          1.          0.        ]
 [13.71755525 -0.01433737  0.01433737]
 [ 0.          0.0877193  -0.0877193 ]]
B=
[[ 0.   ]
 [-0.251]
 [ 0.251]]
Q=
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
R=
[[1]]
P=
[[1782.87869324  480.63198435   23.12004015]
 [ 480.63198435  129.7156499     6.29651464]
 [  23.12004015    6.29651464    3.13121384]]
K=
[[-114.83549799  -30.97820295   -0.7944905 ]]


# 5. 仿真游戏
<kbd>A</kbd>:动量轮逆时针旋转  
<kbd>D</kbd>:动量轮顺时针旋转  
<kbd>W</kbd>:启动LQR控制器  
<kbd>S</kbd>:动量轮刹车  

In [28]:
theta_b_d_d = 0
theta_b_d = 0
theta_b = 0
theta_w_d_d = 0
theta_w_d = 0
theta_w = 0
u = 0
import numpy as np

def acc_cal():
    global theta_b_d_d
    global theta_w_d_d
    T_m = K_m * u
    theta_b_d_d = ((m_b*l_b+m_w*l)*g*np.sin(theta_b)-T_m-C_w*(theta_b_d-theta_w_d))/(I_b+m_w*l**2)
    theta_w_d_d = (T_m-C_w*(theta_w_d-theta_b_d))/I_w
    # print("acc:",theta_b_d_d,theta_w_d_d,end='\r')

def vel_cal():
    global theta_b_d
    global theta_w_d
    theta_b_d += theta_b_d_d*dt
    theta_w_d += theta_w_d_d*dt
    if(-0.01<abs(theta_b_d)<0.01):
        theta_b_d=0
    if(-0.01<abs(theta_w_d)<0.01):
        theta_w_d=0
    theta_w_d=np.clip(theta_w_d,-theta_w_d_max,theta_w_d_max)
    # print("vel",theta_b_d,theta_w_d,end='\r')

def pos_cal():
    global theta_b_d_d
    global theta_w_d_d
    global theta_b_d
    global theta_w_d
    global theta_b
    global theta_w
    theta_b += theta_b_d*dt
    theta_w += theta_w_d*dt
    # print("pos",theta_b,theta_w,end='\r')
    if(theta_b > np.pi/4):
        # theta_b=2*np.pi/4-theta_b
        theta_b = np.pi/4
        theta_b_d_d = (-theta_b_d*elastic_coeff - theta_b_d)/dt
        theta_b_d = -theta_b_d*elastic_coeff
    elif(theta_b < -np.pi/4):
        # theta_b=-2*np.pi/4-theta_b
        theta_b=-np.pi/4
        theta_b_d_d = (-theta_b_d*elastic_coeff - theta_b_d)/dt
        theta_b_d = -theta_b_d*elastic_coeff

def brake():
    global theta_b_d_d
    global theta_w_d_d
    global theta_b_d
    global theta_w_d
    theta_b_d = (theta_b_d*(I_b+m_w*l**2)+theta_w_d*(I_w))/(I_b+I_w+m_w*l**2)
    theta_w_d = theta_b_d


def cal():
    acc_cal()
    vel_cal()
    pos_cal()
    print("acc:",theta_b_d_d,theta_w_d_d,end='\t')
    print("vel",theta_b_d,theta_w_d,end='\t')
    print("pos",theta_b,theta_w,end='\r\b\r\b\r')


In [29]:
import pygame as pg
import numpy as np


def rotate(surface, angle, pivot, offset):
    """Rotate the surface around the pivot point.

    Args:
        surface (pygame.Surface): The surface that is to be rotated.
        angle (float): Rotate by this angle.
        pivot (tuple, list, pygame.math.Vector2): The pivot point.
        offset (pygame.math.Vector2): This vector is added to the pivot.
    """
    rotated_image = pg.transform.rotozoom(surface, -angle, 1)  # Rotate the image.
    rotated_offset = offset.rotate(angle)  # Rotate the offset vector.
    # Add the offset vector to the center/pivot point to shift the rect.
    rect = rotated_image.get_rect(center=pivot+rotated_offset)
    return rotated_image, rect  # Return the rotated image and shifted rect.


pg.init()
screen = pg.display.set_mode((320, 320))
clock = pg.time.Clock()
BG_COLOR = pg.Color('gray12')
# The original image will never be modified.
IMAGE = pg.Surface((150, 150), pg.SRCALPHA)
pg.draw.polygon(IMAGE, pg.Color('dodgerblue3'), ((0, 0), (150, 0), (150, 150),(0, 150)))
pg.draw.circle(IMAGE, pg.Color('white'),(75,75),50)

# Store the original center position of the surface.
pivot = [screen.get_width()/2, screen.get_height()]
# This offset vector will be added to the pivot point, so the
# resulting rect will be blitted at `rect.topleft + offset`.
offset = pg.math.Vector2(75, -75)
# 初始化字体对象
font = pg.font.Font(None, 30)
angle = -45

n=0

running = True
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False

    keys = pg.key.get_pressed()
    
    u = 0
    if keys[pg.K_d] or keys[pg.K_RIGHT]:
        u += -1
    elif keys[pg.K_a] or keys[pg.K_LEFT]:
        u += 1
    if keys[pg.K_s]:
        brake()
    if keys[pg.K_w]:
        u = LQR_controller([theta_b,theta_b_d,theta_w_d])

    cal()
    n+=1
    angle = -45 - theta_b*180/np.pi
    # Rotated version of the image and the shifted rect.
    rotated_image, rect = rotate(IMAGE, angle, pivot, offset)

    # Drawing.
    screen.fill(BG_COLOR)
    screen.blit(rotated_image, rect)  # Blit the rotated image.
    pg.draw.circle(screen, (30, 250, 70), pivot, 3)  # Pivot point.
    pg.draw.rect(screen, (30, 250, 70), rect, 1)  # The rect.

    pg.draw.line(screen ,pg.Color('red'), rect.center, np.array(rect.center)+[np.sin(theta_w)*-50,np.cos(-theta_w)*-50], 2)

    pg.display.set_caption('Angle: {}'.format(angle))

    # 计算并显示FPS
    fps_text = font.render("FPS: {:.2f}".format(clock.get_fps()), True, (255, 255, 255))
    screen.blit(fps_text, (10, 10))

    pg.display.flip()
    clock.tick(fps)
pg.quit()